In [ ]:
import os
import json
import spacy
import faiss
from RAGLibrary import Widgets, Define
from RAGLibrary import SegmentChunks
from RAGLibrary import ExtractData2 as Extract

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
exceptions = {
    "a", "an", "the",
    "and", "but", "or", "nor", "for", "so", "yet",
    "at", "by", "in", "of", "on", "to", "from", "with", "as",
    "into", "like", "over", "under", "up", "down", "out", "upon", "onto",  
    "amid", "among", "between", "before", "after", "against"  
}

In [ ]:
widgets_list = Widgets.create_name_form()

In [ ]:
config = Define.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
chunks_base = config["chunks_base"]
json_file_path = config["json_file_path"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

SEARCH_ENGINE = faiss.IndexFlatIP

In [ ]:
path = dcmt_path
Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None
print(Contents)

In [ ]:
# EXTRACT DATA
text_data = Extract.extracted(dcmt_path)
text_data

BASE


In [ ]:
# FORMAT TEXT
def format_text(text, case="upper"):
    if case == "upper":
        return text.upper()
    elif case == "Chapter":
        return ' '.join(word.capitalize() if word.lower() not in exceptions else word.lower() for word in text.split())
    return text

In [ ]:
# SIZE OF Chapter AND Title
def find_sizes(text_data):
    Chapter_size, sub_size = 0, 0
    
    for entry in text_data:
        text, font_size = entry["text"], entry["font_size"]
        if text.isupper():
            Chapter_size = max(Chapter_size, font_size)
        else:
            sub_size = max(sub_size, font_size)
    return Chapter_size, sub_size

In [ ]:
# ADD CHUNKS
def add_chunk(chunks, content):
    if content[Contents]:
        chunks.append(content.copy())
        content[Contents] = []

In [ ]:
# MAIN FUNCTION
def main(text_data, Chapter_size, sub_size):
    chunks = []
    content = {"Chapter": None, "Title": None, "Article": None, Contents: []}

    for entry in text_data:
        text, font_size = entry["text"], entry["font_size"]
        if not text:
            continue
        if text.isupper():
            if font_size == Chapter_size:
                add_chunk(chunks, content)
                content["Chapter"] = format_text(text, "upper")
                content["Title"] = None
                content["Article"] = None
            else:
                add_chunk(chunks, content)
                content["Article"] = format_text(text, "upper")
        else:
            if font_size == sub_size:
                add_chunk(chunks, content)
                content["Title"] = format_text(text, "Chapter")
                content["Article"] = None
            else:
                if content[Contents]:
                    last_sentence = content[Contents][-1]
                    if not last_sentence.endswith((".", "!", "?")):
                        content[Contents][-1] += " " + text
                    else:
                        content[Contents].append(text)
                else:
                    content[Contents].append(text)
    
    add_chunk(chunks, content)
    return chunks

In [ ]:
# CHUNKS BASE
Chapter_size, sub_size = find_sizes(text_data)
chunks = main(text_data, Chapter_size, sub_size)

In [ ]:
# EXPORT BASE
with open(chunks_base, "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=4, ensure_ascii=False)

print(f"Base data saved to {chunks_base}")

FINAL


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
SegmentChunks.process_json(chunks_base, json_file_path, Contents, WORD_LIMIT, nlp)